Import the packages and set the directory.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from kerastuner.tuners import RandomSearch
import os
# Define data paths
train_dir = 'train'
val_dir = 'valid'
test_dir = 'test'

C:\Users\ACER\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using TensorFlow backend


C:\Users\ACER\AppData\Local\Temp\ipykernel_2520\56100754.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Set training, validilating and testing data. Each image has 112 x 112 resolution.  
The dataset can be found at: https://www.kaggle.com/datasets/gpiosenka/100-bird-species

In [5]:
# Define the ImageDataGenerator for training and validation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)  

# Define the training, validation, and test generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(112, 112),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    val_dir,  # Corrected to val_dir
    target_size=(112, 112),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(  # Added test generator
    test_dir,
    target_size=(112, 112),
    batch_size=32,
    class_mode='categorical'
)

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


Define CNNs by the implemention of VGG16, ResNet50 and MobileNetV2.
Reduce learning is set.
Each architecture has the following parameters:
+ the pre-trained base, a flattening layer, a dense layer with 512 units
+ batch normalization, ReLU activation, a dropout layer for regularization
+ the final classification layer with 525 units (representing the bird species) 
+ softmax activation for probability distribution. 
+ Adam optimizer
+ a learning rate of  10^{-3}
+ categorical cross-entropy loss function. 

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard

# Reduce learning rate when a metric has stopped improving.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-3)

# Stop training when a monitored quantity has stopped improving.
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

def build_vgg_model():
    base_model = tf.keras.applications.VGG16(weights='imagenet', input_shape=(112, 112, 3), include_top=False)
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=l2(1e-4)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        layers.Dense(525),
        layers.BatchNormalization(),
        layers.Activation('softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def build_resnet_model():
    base_model = tf.keras.applications.ResNet50(weights='imagenet', input_shape=(112, 112, 3), include_top=False)
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=l2(1e-4)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        layers.Dense(525),
        layers.BatchNormalization(),
        layers.Activation('softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def build_mobilenet_model():
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(112, 112, 3), include_top=False)
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=l2(1e-4)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        layers.Dense(525),
        layers.BatchNormalization(),
        layers.Activation('softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


Train each model with epoch 10. 
+ early stop is set to reduc computional burden
+ Tensorboard is set to monitor loss and accuracy.

In [19]:
# Train VGG model
vgg_model = build_vgg_model()
vgg_checkpoint = ModelCheckpoint(filepath='best_model_vgg_n.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)
vgg_tensorboard = TensorBoard(log_dir=os.path.join(os.getcwd(), 'logs/vgg_n'))
vgg_history = vgg_model.fit(train_generator, validation_data=valid_generator, epochs=10, callbacks=[early_stopping, vgg_checkpoint, vgg_tensorboard, reduce_lr])

Epoch 1/10
2645/2645 [==============================] - ETA: 0s - loss: 4.9597 - accuracy: 0.1307
Epoch 1: val_loss improved from inf to 3.32571, saving model to best_model_vgg_n.h5
2645/2645 [==============================] - 2584s 976ms/step - loss: 4.9597 - accuracy: 0.1307 - val_loss: 3.3257 - val_accuracy: 0.3806 - lr: 0.0010
Epoch 2/10


C:\Users\ACER\anaconda3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2645/2645 [==============================] - ETA: 0s - loss: 4.3793 - accuracy: 0.2099
Epoch 2: val_loss improved from 3.32571 to 2.98703, saving model to best_model_vgg_n.h5
2645/2645 [==============================] - 2311s 874ms/step - loss: 4.3793 - accuracy: 0.2099 - val_loss: 2.9870 - val_accuracy: 0.4389 - lr: 0.0010
Epoch 3/10
2645/2645 [==============================] - ETA: 0s - loss: 4.1969 - accuracy: 0.2330
Epoch 3: val_loss improved from 2.98703 to 2.79049, saving model to best_model_vgg_n.h5
2645/2645 [==============================] - 2365s 894ms/step - loss: 4.1969 - accuracy: 0.2330 - val_loss: 2.7905 - val_accuracy: 0.4636 - lr: 0.0010
Epoch 4/10
2645/2645 [==============================] - ETA: 0s - loss: 4.0754 - accuracy: 0.2489
Epoch 4: val_loss improved from 2.79049 to 2.68553, saving model to best_model_vgg_n.h5
2645/2645 [==============================] - 2584s 977ms/step - loss: 4.0754 - accuracy: 0.2489 - val_loss: 2.6855 - val_accuracy: 0.4796 - lr: 0.0010


In [20]:
# Train ResNet model
resnet_model = build_resnet_model()
resnet_checkpoint = ModelCheckpoint(filepath='best_model_resnet_n.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)
resnet_tensorboard = TensorBoard(log_dir=os.path.join(os.getcwd(), 'logs/resnet_n'))
resnet_history = resnet_model.fit(train_generator, validation_data=valid_generator, epochs=10, callbacks=[early_stopping, resnet_checkpoint, resnet_tensorboard, reduce_lr])

Epoch 1/10
2645/2645 [==============================] - ETA: 0s - loss: 6.0603 - accuracy: 0.0148
Epoch 1: val_loss improved from inf to 6.44562, saving model to best_model_resnet_n.h5
2645/2645 [==============================] - 1997s 753ms/step - loss: 6.0603 - accuracy: 0.0148 - val_loss: 6.4456 - val_accuracy: 0.0156 - lr: 0.0010
Epoch 2/10
2645/2645 [==============================] - ETA: 0s - loss: 5.8158 - accuracy: 0.0265
Epoch 2: val_loss improved from 6.44562 to 5.91726, saving model to best_model_resnet_n.h5
2645/2645 [==============================] - 1660s 627ms/step - loss: 5.8158 - accuracy: 0.0265 - val_loss: 5.9173 - val_accuracy: 0.0309 - lr: 0.0010
Epoch 3/10
2645/2645 [==============================] - ETA: 0s - loss: 5.7262 - accuracy: 0.0303
Epoch 3: val_loss did not improve from 5.91726
2645/2645 [==============================] - 1588s 600ms/step - loss: 5.7262 - accuracy: 0.0303 - val_loss: 6.5004 - val_accuracy: 0.0145 - lr: 0.0010
Epoch 4/10
2645/2645 [======

In [21]:
# Train MobileNet model
mobilenet_model = build_mobilenet_model()
mobilenet_checkpoint = ModelCheckpoint(filepath='best_model_mobilenet_n.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)
mobilenet_tensorboard = TensorBoard(log_dir=os.path.join(os.getcwd(), 'logs/mobilenet_n'))
mobilenet_history = mobilenet_model.fit(train_generator, validation_data=valid_generator, epochs=10, callbacks=[early_stopping, mobilenet_checkpoint, mobilenet_tensorboard, reduce_lr])

Epoch 1/10
2645/2645 [==============================] - ETA: 0s - loss: 4.2990 - accuracy: 0.2730
Epoch 1: val_loss improved from inf to 2.55653, saving model to best_model_mobilenet_n.h5
2645/2645 [==============================] - 716s 269ms/step - loss: 4.2990 - accuracy: 0.2730 - val_loss: 2.5565 - val_accuracy: 0.5714 - lr: 0.0010
Epoch 2/10
2645/2645 [==============================] - ETA: 0s - loss: 3.8521 - accuracy: 0.3551
Epoch 2: val_loss improved from 2.55653 to 2.39618, saving model to best_model_mobilenet_n.h5
2645/2645 [==============================] - 696s 263ms/step - loss: 3.8521 - accuracy: 0.3551 - val_loss: 2.3962 - val_accuracy: 0.6053 - lr: 0.0010
Epoch 3/10
2645/2645 [==============================] - ETA: 0s - loss: 3.7255 - accuracy: 0.3752
Epoch 3: val_loss improved from 2.39618 to 2.37381, saving model to best_model_mobilenet_n.h5
2645/2645 [==============================] - 702s 265ms/step - loss: 3.7255 - accuracy: 0.3752 - val_loss: 2.3738 - val_accuracy

Open Tensorboard to monitor loss and accuracy during the training

In [23]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9300), started 16:30:38 ago. (Use '!kill 9300' to kill it.)

The trained models for each architectures are loaded and utilised to predict bird species of the test data to obtain the accuracies.

In [22]:
from tensorflow.keras.models import load_model
import os

# Define the architecture names
architectures = ['vgg', 'resnet', 'mobilenet']

# Load the best models for each architecture
models = {}
for arch in architectures:
    model_path = f'best_model_{arch}_n.h5'
    if os.path.exists(model_path):  # Check if the model file exists
        models[arch] = load_model(model_path)
    else:
        print(f"Model file for {arch} not found!")

# Evaluate each model on the test set
accuracies = {}
for arch, model in models.items():
    loss, acc = model.evaluate(test_generator, verbose=0)
    accuracies[arch] = acc

accuracies

{'vgg': 0.5580952167510986,
 'resnet': 0.0476190485060215,
 'mobilenet': 0.6666666865348816}